En este notebook encontrarán los pasos e instrucciones necesarias para para realizar el tratamiento de los datos.

## **Description**
Identificador | Description
---|---
id | Identificación de la plataforma con la película
show_id | Identificación de la película
title | Título de la película
director | Director de la película
cast | Actores de reparto
country | País de la procedencia de la Película
date_added | Fecha de agregada a la plataforma
release_year | Año de lanzamiento
rating | calificación
duration | Duración de la película
listed_in | Lista de tipo de película
description | Descripción de la película
plataforma | Tìpo de plataforma
duration_int | Duración de la película (entero)
duration_type | Tipo de duración de la película

In [35]:
# Importar librerías de python
import pandas as pd
import numpy as np

### Extracting data

In [2]:
# Cargar los dataset y crear dataframes
df_a = pd.read_csv("datasets/amazon_prime_titles.csv", sep= ",")
df_d = pd.read_csv("datasets/disney_plus_titles.csv", sep= ",")
df_h = pd.read_csv("datasets/hulu_titles.csv", sep= ",")
df_n = pd.read_csv("datasets/netflix_titles.csv", sep= ",")

In [3]:
# Observamos los dataframe 
df_a.head(2)
# df_d.head(2)
# df_h.head(2)
# df_n.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...


### Transformación

#### Plataformas

1. `Generar campo id:` Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123).

In [4]:
#Crear la columna con la generación del "id"
df_a["id"] = "a" + df_a["show_id"].astype(str)
df_d["id"] = "d" + df_d["show_id"].astype(str)
df_h["id"] = "h" + df_h["show_id"].astype(str)
df_n["id"] = "n" + df_n["show_id"].astype(str)

In [5]:
# Generar una columna plataforma
df_a["platform"] = "amazon" 
df_d["platform"] = "disney" 
df_h["platform"] = "hulu" 
df_n["platform"] = "netflix" 

In [6]:
# Concatenar todos los dataframe con la función "concat"
df_plataformas = pd.concat([df_a, df_d, df_h, df_n], ignore_index= True)

2.	Los `valores nulos` del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”.)

In [7]:
# reemplazar los valores nulos en la columna "rating" con un nuevo valor "G" 
df_plataformas["rating"] = df_plataformas["rating"].fillna('G')

3. De haber fechas, deberán tener el `formato AAAA-mm-dd`

In [8]:
# convertir la columna "date_added" a tipo datetime
df_plataformas["date_added"] = pd.to_datetime(df_plataformas["date_added"])

4.	Los `campos de texto` deberán estar en `minúsculas`, sin excepciones

In [9]:
df_plataformas = df_plataformas.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

In [10]:
df_plataformas.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,amazon
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,amazon


5.	El campo `duration` debe `convertirse en dos campos: duration_int y duration_type`. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [11]:
# Con el método "str.split()" podemos separar la columna "duration" en dos columnas "duration_int" y "duration_type"
df_plataformas[["duration_int","duration_type"]] = df_plataformas["duration"].str.split(" ", expand = True)

# Cambiamos el tipo de dato de 'duration_int' a int
df_plataformas["duration_int"] = df_plataformas["duration_int"].astype("Int64")

#### Rating

In [14]:
# Se cargan el dataset rating
df_r1 = pd.read_csv("ratings/1.csv")
df_r2 = pd.read_csv("ratings/2.csv")
df_r3 = pd.read_csv("ratings/3.csv")
df_r4 = pd.read_csv("ratings/4.csv")
df_r5 = pd.read_csv("ratings/5.csv")
df_r6 = pd.read_csv("ratings/6.csv")
df_r7 = pd.read_csv("ratings/7.csv")
df_r8 = pd.read_csv("ratings/8.csv")

In [15]:
# Concatenando todos los dataframe con la función "concat"
df_rating = pd.concat([df_r1, df_r2, df_r3,df_r4, df_r5, df_r6, df_r7, df_r8], ignore_index= True)

De haber fechas, deberán tener el `formato AAAA-mm-dd`

In [16]:
# Convertir la columna 'fechas' al formato AAAA-mm-dd
df_rating["timestamp"] = pd.to_datetime(df_rating["timestamp"], unit="s").dt.strftime('%Y-%m-%d')

# convertir la columna "date_added" a tipo datetime
df_rating["timestamp"] = pd.to_datetime(df_rating["timestamp"])

In [17]:
# Renombrar los nombres de las columnas 
df_rating = df_rating.rename(columns={"movieId": "id", "rating": "scored"})

In [20]:
df_rating.head(2)

,userId,scored,timestamp,id
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186


In [24]:
# Exportamos el dataframe df_plataformas en formato csv
df_rating.to_csv('df_rating.csv', index=False)

### Promedio de score

In [25]:
# Obtenemos el promedio de rating utilizando el método groupby
prom_rating = df_rating.groupby("id")["scored"].mean().round(2)

In [27]:
# Convertirmos a dataframe
prom_rating = pd.DataFrame({"id":prom_rating.index, "scored": prom_rating.values})

In [34]:
prom_rating.head(2)

,id,scored
0,as1,3.47
1,as10,3.44


### Datasets para la API

In [29]:
# Unimos los dataframes verticalmente utilizando el método merge 
df_plataformas = pd.merge(df_plataformas,prom_rating, on = "id")

In [31]:
# Exportamos el dataframe df_plataformas en formato csv
df_plataformas.to_csv('df_plataformas.csv', index=False)